<a href="https://colab.research.google.com/github/ameliamacedogithub/repositorionaoseremoseliminados/blob/main/Desafio4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nome do grupo:** Não seremos eliminados

Participantes:

*   João Souza
*   Maria Amélia França de Macedo
*   Rodrigo Noll
*   Sérgio Noll


Pipeline automatizado para cálculo de VR/VA
- Leitura das bases
- Padronização (nomes, datas, chaves)
- EDA
- Regras de exclusão
- Cálculo de dias úteis elegíveis
- Regra de desligamento (dia 15)
- Cálculo de VR (80% empresa / 20% colaborador)
- Geração do layout final (igual à "VR MENSAL 05.2025")


Instalação de pré-requisitos


In [ ]:
!pip -q install gdown
!pip -q install pandas openpyxl xlsxwriter



Importação de Bibliotecas

In [ ]:
from __future__ import annotations
import os, re, zipfile, unicodedata,tempfile,gc
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
import pandas as pd
import numpy as np
import gdown
from google.colab import files
from datetime import datetime
from zoneinfo import ZoneInfo
import ipywidgets as widgets
from IPython.display import display, clear_output
import holidays






Baixar e extrair ZIP do Google Drive (Colab)
- Usa gdown (lida com confirmações do Drive)
- Extrai para uma pasta temporária
- Lista o que foi extraído


Se o link estiver quebrado irá solicitar usuário enviar arquivo zip os as planilhas, baixar e extrair ZIP (se for o caso)


In [ ]:
FILE_ID: Optional[str] = "1SzLGxIRo2j8eHPdqe06xXxtoGGrWHoJ5"  # ex: "1SzLGxIRo2j8eHPdqe06xXxtoGGrWHoJ5"
DRIVE_URL: Optional[str] = 'https://drive.google.com/file/d/1SzLGxIRo2j8eHPdqe06xXxtoGGrWHoJ5/view?usp=drive_link'            # ex: "https://drive.google.com/file/d/ABC123/view?usp=sharing"

# Pasta onde os arquivos ficarão disponíveis
DEST_DIR = Path("/content/dados_vrva")
DEST_DIR.mkdir(parents=True, exist_ok=True)

# Extensões consideradas úteis
VALID_EXTS = {".xlsx", ".xls", ".csv"}

# ====================== FUNÇÕES UTILITÁRIAS ======================
def in_colab() -> bool:
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False

def ensure_gdown():
    try:
        import gdown  # noqa: F401
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "gdown"])

def extract_drive_id(url: str) -> Optional[str]:
    pats = [
        r"/d/([a-zA-Z0-9_-]{20,})/",
        r"id=([a-zA-Z0-9_-]{20,})",
        r"/file/u/\d+/d/([a-zA-Z0-9_-]{20,})/",
    ]
    for p in pats:
        m = re.search(p, url)
        if m:
            return m.group(1)
    return None

def download_zip_from_drive(file_id: Optional[str]=None, url: Optional[str]=None) -> Path:
    """Baixa um ZIP do Drive (via FILE_ID ou URL) para uma pasta temporária e retorna o path do arquivo ZIP."""
    ensure_gdown()
    import gdown

    if url and not file_id:
        file_id = extract_drive_id(url)

    if not file_id and not url:
        raise ValueError("Nem FILE_ID nem DRIVE_URL foram informados.")

    tmp_dir = Path(tempfile.mkdtemp(prefix="vrva_zip_"))
    zip_path = tmp_dir / "desafio_vrva.zip"
    print(f"Baixando ZIP para: {zip_path}")

    if file_id:
        gdown.download(id=file_id, output=str(zip_path), quiet=False)
    else:
        gdown.download(url=url, output=str(zip_path), quiet=False)

    if not zip_path.exists() or zip_path.stat().st_size == 0:
        raise RuntimeError("Download falhou ou retornou arquivo vazio.")
    return zip_path

def safe_extract_zip(zip_path: Path, dest_dir: Path) -> Tuple[List[str], List[Tuple[str, str]]]:
    """Extrai ZIP de forma tolerante. Retorna (ok, bad) com nomes e erros."""
    ok, bad = [], []
    with zipfile.ZipFile(zip_path, "r") as z:
        for info in z.infolist():
            try:
                z.extract(info, dest_dir)
                ok.append(info.filename)
            except Exception as e:
                bad.append((info.filename, str(e)))
    return ok, bad

def inventory(dir_path: Path) -> pd.DataFrame:
    rows = []
    for p in dir_path.rglob("*"):
        if p.is_file():
            rows.append({
                "relative_path": str(p.relative_to(dir_path)),
                "size_kb": round(p.stat().st_size/1024, 1),
                "ext": p.suffix.lower()
            })
    if rows:
        df = pd.DataFrame(rows).sort_values(["ext","relative_path"]).reset_index(drop=True)
    else:
        df = pd.DataFrame(columns=["relative_path","size_kb","ext"])
    return df

def has_valid_files(dir_path: Path, valid_exts: set) -> bool:
    return any(p.is_file() and p.suffix.lower() in valid_exts for p in dir_path.rglob("*"))

def prompt_upload_fallback(dest_dir: Path) -> None:
    """Solicita upload de ZIP ou planilhas individuais quando em Colab."""
    if not in_colab():
        raise RuntimeError(
            "Falha no download/extração e este ambiente não é Google Colab. "
            "Suba os arquivos manualmente para a pasta: {}".format(dest_dir)
        )
    from google.colab import files  # type: ignore
    print("\n⚠️ Não foi possível obter planilhas válidas do Drive.")
    print("Por favor, faça upload de um arquivo .zip OU de um ou mais .xlsx/.csv.")
    uploaded = files.upload()  # dict {filename: bytes}

    # Se for um único ZIP, extrai. Se forem planilhas soltas, apenas grava no destino.
    if len(uploaded) == 1:
        name, data = next(iter(uploaded.items()))
        if name.lower().endswith(".zip"):
            up_zip_path = dest_dir / name
            with open(up_zip_path, "wb") as f:
                f.write(data)
            print(f"Arquivo ZIP recebido: {up_zip_path}")
            ok, bad = safe_extract_zip(up_zip_path, dest_dir)
            print(f"✅ Extraídos: {len(ok)} | ⚠️ Erros: {len(bad)}")
            if bad:
                for m, e in bad[:20]:
                    print(f" - {m} -> {e}")
        else:
            # Não é zip: salva como planilha solta
            out_path = dest_dir / name
            with open(out_path, "wb") as f:
                f.write(data)
            print(f"Arquivo recebido: {out_path}")
    else:
        # Múltiplos arquivos
        for name, data in uploaded.items():
            out_path = dest_dir / name
            with open(out_path, "wb") as f:
                f.write(data)
        print(f"{len(uploaded)} arquivos recebidos e gravados em: {dest_dir}")

# ====================== PIPELINE PRINCIPAL ======================
def obter_planilhas(dest_dir: Path = DEST_DIR) -> Path:
    """
    Tenta baixar e extrair um ZIP do Drive.
    Se falhar (ou se não houver planilhas válidas), abre o fluxo de upload manual.
    Retorna a pasta contendo os arquivos prontos para uso.
    """
    # 1) Tenta via Drive (se configurado)
    drive_attempted = False
    if FILE_ID or DRIVE_URL:
        drive_attempted = True
        try:
            zip_path = download_zip_from_drive(FILE_ID, DRIVE_URL)
            print(f"Pasta de extração: {dest_dir}")
            ok, bad = safe_extract_zip(zip_path, dest_dir)
            print(f"\nArquivos extraídos com sucesso: {len(ok)}")
            print(f"Arquivos com erro: {len(bad)}")
            if bad:
                print("\n⚠️ Houve entradas com erro na extração:")
                for m, e in bad[:20]:
                    print(f" - {m} -> {e}")
                if len(bad) > 20:
                    print(f"... e mais {len(bad)-20} itens")

        except Exception as e:
            print(f"\n❌ Falha ao baixar/extrair do Drive: {e}")

    # 2) Verifica se há arquivos válidos; se não, pede upload
    if not has_valid_files(dest_dir, VALID_EXTS):
        if drive_attempted:
            print("\nNenhuma planilha válida encontrada após tentar o Drive.")
        try:
            prompt_upload_fallback(dest_dir)
        except Exception as e:
            raise RuntimeError(f"Falha no fallback de upload: {e}")

    # 3) Inventário final
    df = inventory(dest_dir)
    print("\nInventário dos arquivos disponíveis:")
    try:
        from IPython.display import display  # type: ignore
        display(df)
    except Exception:
        print(df.to_string(index=False))

    # 4) Checagem final
    if not has_valid_files(dest_dir, VALID_EXTS):
        raise RuntimeError("Nenhuma planilha .xlsx/.xls/.csv válida encontrada mesmo após upload.")

    print(f"\n✅ Arquivos prontos em: {dest_dir}")
    return dest_dir

if __name__ == "__main__":
     obter_planilhas()





In [ ]:
DIA_CORTE = 15


# mês corrente
now = datetime.now(ZoneInfo("America/Sao_Paulo"))
ANO_ALVO = 2025
limite = now.month if now.year == ANO_ALVO else 12
meses = list(range(limite, 0, -1))

opcoes = [f"{m:02d}/{ANO_ALVO}" for m in meses]
dd = widgets.Dropdown(options=opcoes, value=opcoes[0], description="Competência:")

btn = widgets.Button(description="Confirmar", button_style="primary")
out = widgets.Output()

def on_click(_):
    global MES_REF, ANO_REF, DIA_CORTE   # <--- tornam-se globais
    with out:
        clear_output()
        comp = dd.value
        MES_REF, ANO_REF = comp.split("/")
        MES_REF, ANO_REF = int(MES_REF), int(ANO_REF)
        print(f"Competência selecionada: {comp}")
        print(f"MES_REF={MES_REF}, ANO_REF={ANO_REF}, DIA_CORTE={DIA_CORTE}")

display(widgets.VBox([dd, btn, out]))
btn.on_click(on_click)


In [ ]:
#Caminhos de arquivos
base_path = "/content/dados_vrva"
adm_path = os.path.join(base_path, "ADMISSÃO ABRIL.xlsx")
afa_path = os.path.join(base_path, "AFASTAMENTOS.xlsx")
apr_path = os.path.join(base_path, "APRENDIZ.xlsx")
ati_path = os.path.join(base_path, "ATIVOS.xlsx")
dias_path = os.path.join(base_path, "Base dias uteis.xlsx")
sind_path = os.path.join(base_path, "Base sindicato x valor.xlsx")
des_path = os.path.join(base_path, "DESLIGADOS.xlsx")
est_path = os.path.join(base_path, "ESTÁGIO.xlsx")
ext_path = os.path.join(base_path, "EXTERIOR.xlsx")
fer_path = os.path.join(base_path, "FÉRIAS.xlsx")
vr_path  = os.path.join(base_path, "VR MENSAL 05.2025.xlsx")

#TODO: checar se as planilhas carregadas correspondem a competência selecionada

In [ ]:
# ---------- helpers ----------


def strip_accents(s: str) -> str:
    return ''.join(ch for ch in unicodedata.normalize('NFKD', str(s)) if not unicodedata.combining(ch))

def norm_col(s: str) -> str:
    """Normaliza nome de coluna para matching: sem acento, minúscula, sem pontuação, espaço único."""
    s = strip_accents(str(s)).lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^a-z0-9 ]", "", s)
    return s

def looks_unnamed_token(x) -> bool:
    sx = str(x) if x is not None else ""
    sx_strip = sx.strip()
    return (sx_strip == "") or norm_col(sx_strip).startswith("unnamed")

def is_numeric_like(x) -> bool:
    sx = str(x).strip()
    if sx == "" or sx.lower() == "nan":
        return False
    try:
        float(sx.replace(",", "."))
        return True
    except Exception:
        return False


#TODO: retirar o critério de keywords#

def choose_header_row(df_nohdr: pd.DataFrame, max_scan: int = 15) -> int:
    """
    Escolhe a melhor linha para virar cabeçalho.
    Critérios (maior score é melhor):
      +5 se contém keywords (matric, admiss, cargo, funcao)
      +2 se contém letras (A–Z)
      -1 se vazio/'unnamed'
      -0.5 se numérico
    Também penaliza linhas muito vazias.
    """
    nrows = min(max_scan, len(df_nohdr))
    best_idx, best_score = 0, -1e9
    keywords = ("matric", "admiss", "cargo", "funcao", "funçao", "função", "funca")
    for i in range(nrows):
        row = df_nohdr.iloc[i]
        score = 0.0
        empties = 0
        for v in row:
            sv = str(v)
            nk = norm_col(sv)
            if nk == "" or nk.startswith("unnamed"):
                score -= 1.0
                empties += 1
            elif any(k in nk for k in keywords):
                score += 5.0
            elif re.search(r"[a-zA-Z]", sv):
                score += 2.0
            elif is_numeric_like(sv):
                score -= 0.5
        score -= empties * 0.2
        if score > best_score:
            best_score, best_idx = score, i
    return best_idx

def load_vr_with_header_detection(path: str) -> pd.DataFrame:
    """
    Lê a planilha VR e tenta detectar automaticamente a linha de cabeçalho.
    Retorna um DataFrame com colunas definidas pela linha escolhida e dados a partir da próxima linha.
    """
    # Lemos sem header para ter acesso a todas as linhas "cruas"
    df_all = pd.read_excel(path, engine="openpyxl", header=None)
    if df_all.empty:
        return df_all

    hdr_idx = choose_header_row(df_all)
    header_vals = list(df_all.iloc[hdr_idx].astype(str))

    # Conserta nomes vazios/duplicados com rótulos "col_i"
    fixed = []
    seen = set()
    for j, v in enumerate(header_vals):
        name = v.strip()
        if name == "" or looks_unnamed_token(name) or is_numeric_like(name):
            name = f"col_{j+1}"
        # garante unicidade
        base, k, candidate = name, 1, name
        while candidate in seen:
            k += 1
            candidate = f"{base}_{k}"
        seen.add(candidate)
        fixed.append(candidate)

    df = df_all.iloc[hdr_idx + 1:].copy()
    df.columns = fixed
    # Remove colunas completamente vazias
    df = df.loc[:, ~(df.isna().all(axis=0))]
    df = df.reset_index(drop=True)
    return df

def looks_unnamed_header(c) -> bool:
    cs = "" if c is None else str(c)
    return (cs.strip() == "") or cs.startswith("Unnamed")


def find_col(df, *needles):
    """Encontra a primeira coluna de df cujo nome normalizado contenha qualquer 'needle'."""
    for c in df.columns:
        nk = norm_col(c)
        if any(n in nk for n in needles):
            return c
    return None

def dias_uteis_entre(inicio, fim):
    """Conta dias úteis de 'inicio' a 'fim' (inclusive). Se qualquer for NaT, retorna 0."""
    if pd.isna(inicio) or pd.isna(fim):
        return 0
    if fim < inicio:
        return 0
    return len(pd.bdate_range(inicio, fim))

def proximo_corte(mes_ref: pd.Timestamp, dia_corte: int) -> pd.Timestamp:
    """Retorna a data de corte do mês seguinte ao MES_REF."""
    first_next_month = (MES_REF + pd.offsets.MonthBegin(1)).normalize()
    # garante dia válido (se dia_corte > número de dias do mês, usa o último dia do mês)
    last_day = (first_next_month + pd.offsets.MonthEnd(0)).day
    day = min(DIA_CORTE, last_day)
    return pd.Timestamp(first_next_month.year, first_next_month.month, day)

def corte_atual(mes_ref: pd.Timestamp, dia_corte: int) -> pd.Timestamp:
    """Retorna a data de corte do MES_REF."""
    first_month = pd.Timestamp(ANO_REF, MES_REF, 1)
    last_day = (first_month + pd.offsets.MonthEnd(0)).day
    day = min(DIA_CORTE, last_day)
    return pd.Timestamp(first_month.year, first_month.month, day)

def dias_uteis_no_mes(mes_ref: pd.Timestamp) -> int:
    """Dias úteis (seg–sex) do mês completo de MES_REF."""
    inicio = pd.Timestamp(ANO_REF, MES_REF, 1)
    fim    = inicio + pd.offsets.MonthEnd(0)
    return len(pd.bdate_range(inicio, fim))

def dias_uteis_ate_corte(mes_ref: pd.Timestamp, dia_corte: int) -> int:
    """Dias úteis do mês de MES_REF do dia 1 até DIA_CORTE (inclusivo)."""
    inicio = pd.Timestamp(ANO_REF, MES_REF, 1)
    ultimo_dia = (inicio + pd.offsets.MonthEnd(0)).day
    dc = min(DIA_CORTE, ultimo_dia)
    fim = pd.Timestamp(ANO_REF, MES_REF, dc)
    return len(pd.bdate_range(inicio, fim))

def calculo_dias(desconto: int, mes_ref: pd.Timestamp, dia_corte: int, base: str = "mes") -> int:
    """
    Retorna o total de dias a pagar com base em dias úteis reais.
    base="mes"       -> usa todos os dias úteis do mês de MES_REF
    base="ate_corte" -> usa dias úteis do mês até o DIA_CORTE (inclusivo)
    """
    if base == "ate_corte":
        base_dias = dias_uteis_ate_corte(MES_REF, DIA_CORTE)
    else:
        base_dias = dias_uteis_no_mes(MES_REF)
    return int(max(0, base_dias - int(desconto)))
def feriados_br(years, estado=None, municipais=None):
    """
    years: int ou lista de anos (ex.: 2025 ou [2024, 2025])
    estado: UF (ex.: 'SP', 'RJ', 'GO') para incluir feriados estaduais
    municipais: lista de datas municipais em 'YYYY-MM-DD' (strings) a excluir
    """
    br = holidays.Brazil(years=years, subdiv=estado)  # nacionais + estaduais (se UF)
    feriados = {pd.to_datetime(d).date() for d in br.keys()}
    if municipais:
        feriados |= {pd.to_datetime(d).date() for d in municipais}
    return feriados

def dias_uteis_periodo(inicio, fim, feriados):
    """Conta dias úteis (seg–sex) entre 'inicio' e 'fim' (inclusive), excluindo 'feriados'."""
    if pd.isna(inicio) or pd.isna(fim) or fim < inicio:
        return 0
    bd = pd.bdate_range(inicio, fim)
    return int(sum(d.date() not in feriados for d in bd))


# Liste aqui seus feriados municipais específicos (exemplos fictícios):
FERIADOS_MUNICIPAIS = [
    # "2025-05-20",  # exemplo: feriado municipal (substitua pelas datas reais)
]

UF='SP'

feriados = feriados_br(years=ANO_REF, estado=UF, municipais=FERIADOS_MUNICIPAIS)

# dias úteis do mês inteiro:
inicio_mes = pd.Timestamp(ANO_REF, MES_REF, 1)
fim_mes    = inicio_mes + pd.offsets.MonthEnd(0)
dias_uteis_mes = dias_uteis_periodo(inicio_mes, fim_mes, feriados)

# dias úteis até o corte (inclusivo):
fim_corte = pd.Timestamp(ANO_REF, MES_REF, min(DIA_CORTE, (inicio_mes + pd.offsets.MonthEnd(0)).day))
dias_uteis_ate_corte = dias_uteis_periodo(inicio_mes, fim_corte, feriados)

dias_uteis_mes, dias_uteis_ate_corte

df_vr  = load_vr_with_header_detection(vr_path)   # <<< usa detecção de cabeçalho


In [ ]:
#Tratamento da planilha de admissão

# ---------------- 1) Ler planilhas ----------------
df_adm = pd.read_excel(adm_path, engine="openpyxl")

# ---------------- 2) ADMISSÃO: tratar 'obs' ----------------
# Se houver coluna sem nome, renomeia a primeira para 'obs'
new_cols, inserted = [], False
for c in df_adm.columns:
    if (not inserted) and looks_unnamed_header(c):
        new_cols.append("obs")
        inserted = True
    else:
        new_cols.append(c)
df_adm.columns = new_cols

# Remove linhas onde 'obs' está preenchida e exclui a coluna
if "obs" in df_adm.columns:
    df_adm = df_adm[df_adm["obs"].isna()].drop(columns=["obs"])

# ---------------- 3) VR: construir lookup de nomes EXATOS ----------------
vr_cols = list(df_vr.columns)
vr_lookup = {}
for c in vr_cols:
    key = norm_col(c)
    if key:          # ignora vazios
        vr_lookup.setdefault(key, c)  # mantém o primeiro se houver colisão

# ---------------- 4) Renomear ADMISSÃO e descartar colunas sem equivalente ----------------
mapping = {}
for c in df_adm.columns:
    key = norm_col(c)
    target = vr_lookup.get(key)  # nome exato da VR
    if target:
        mapping[c] = target

df_adm = df_adm.rename(columns=mapping)

# Manter apenas colunas que existem na VR e reordenar na ordem da VR
keep_cols = [c for c in df_vr.columns if c in df_adm.columns]
df_adm = df_adm.loc[:, keep_cols]




In [ ]:
#Tratamento da planilha Afastamentos



# ---------------- 1) Ler planilhas ----------------
df_afa = pd.read_excel(afa_path, engine="openpyxl")

# ---------------- 2) ADMISSÃO: tratar 'obs' ----------------
# Se houver coluna sem nome, renomeia a primeira para 'obs'
new_cols, inserted = [], False
for c in df_afa.columns:
    if (not inserted) and looks_unnamed_header(c):
        new_cols.append("obs")
        inserted = True
    else:
        new_cols.append(c)
df_afa.columns = new_cols


# ---------------- 3) VR: construir lookup de nomes EXATOS ----------------
vr_cols = list(df_vr.columns)
vr_lookup = {}
for c in vr_cols:
    key = norm_col(c)
    if key:          # ignora vazios
        vr_lookup.setdefault(key, c)  # mantém o primeiro se houver colisão

# ---------------- 4) Renomear AFASTAMENTO e manter SOMENTE 'Matricula' ----------------
# Nome exato de 'Matricula' conforme VR
mat_exato = next((c for c in df_vr.columns if norm_col(c) == "matricula"), "Matricula")

# Renomeia colunas de AFASTAMENTO para os nomes exatos da VR quando houver equivalência
mapping = {c: vr_lookup[norm_col(c)] for c in df_afa.columns if norm_col(c) in vr_lookup}
df_afa = df_afa.rename(columns=mapping)

# Garante que a coluna 'Matricula' tenha o nome exato da VR
if mat_exato not in df_afa.columns:
    cand = next((c for c in df_afa.columns if norm_col(c) == "matricula"), None)
    if cand is not None:
        df_afa = df_afa.rename(columns={cand: mat_exato})

# Mantém somente a coluna 'Matricula' (exato conforme VR). Se não existir, cria DF vazio com essa coluna.
if mat_exato in df_afa.columns:
    df_afa = df_afa[[mat_exato]]
else:
    df_afa = pd.DataFrame(columns=[mat_exato])





In [ ]:
#Tratamento da planilha Aprendiz,Estágio e Exterior

#TODO para planilha Exterior - checar se a matrícula estiver na planilha ATIVOS,
#e checar se está Trabalhando. Se estiver Trabalhando, paga, se não estiver exclui.

# ---------------- 1) Ler planilhas ----------------

df_apr = pd.read_excel(apr_path, engine="openpyxl")
df_est = pd.read_excel(est_path, engine="openpyxl")
df_ext = pd.read_excel(ext_path, engine="openpyxl")


# ---------------- 2) tratar 'obs' ----------------
# Se houver coluna sem nome, renomeia a primeira para 'obs'
new_cols, inserted = [], False
for c in df_apr.columns:
    if (not inserted) and looks_unnamed_header(c):
        new_cols.append("obs")
        inserted = True
    else:
        new_cols.append(c)
df_apr.columns = new_cols

new_cols, inserted = [], False
for c in df_est.columns:
    if (not inserted) and looks_unnamed_header(c):
        new_cols.append("obs")
        inserted = True
    else:
        new_cols.append(c)
df_est.columns = new_cols

new_cols, inserted = [], False
for c in df_ext.columns:
    if (not inserted) and looks_unnamed_header(c):
        new_cols.append("obs")
        inserted = True
    else:
        new_cols.append(c)
df_ext.columns = new_cols


# ---------------- 3) VR: construir lookup de nomes EXATOS ----------------
vr_cols = list(df_vr.columns)
vr_lookup = {}
for c in vr_cols:
    key = norm_col(c)
    if key:          # ignora vazios
        vr_lookup.setdefault(key, c)  # mantém o primeiro se houver colisão

# ---------------- 4) Renomear e manter SOMENTE 'Matricula' ----------------
# Nome exato de 'Matricula' conforme VR
mat_exato = next((c for c in df_vr.columns if norm_col(c) == "matricula"), "Matricula")

# Renomeia colunas de APRENDIZ para os nomes exatos da VR quando houver equivalência
mapping = {c: vr_lookup[norm_col(c)] for c in df_apr.columns if norm_col(c) in vr_lookup}
df_apr = df_apr.rename(columns=mapping)

# Renomeia colunas de ESTÁGIO para os nomes exatos da VR quando houver equivalência
mapping = {c: vr_lookup[norm_col(c)] for c in df_est.columns if norm_col(c) in vr_lookup}
df_est = df_est.rename(columns=mapping)


# Garante que a coluna 'Matricula' tenha o nome exato da VR
if mat_exato not in df_apr.columns:
    cand = next((c for c in df_apr.columns if norm_col(c) == "matricula"), None)
    if cand is not None:
        df_apr = df_apr.rename(columns={cand: mat_exato})

if mat_exato not in df_est.columns:
    cand = next((c for c in df_est.columns if norm_col(c) == "matricula"), None)
    if cand is not None:
        df_est = df_est.rename(columns={cand: mat_exato})




# Mantém somente a coluna 'Matricula' (exato conforme VR). Se não existir, cria DF vazio com essa coluna.
if mat_exato in df_apr.columns:
    df_apr = df_apr[[mat_exato]]
else:
    df_apr = pd.DataFrame(columns=[mat_exato])

if mat_exato in df_est.columns:
    df_est = df_est[[mat_exato]]
else:
    df_est = pd.DataFrame(columns=[mat_exato])

# --- Renomear EXTERIOR e manter SOMENTE 'Matricula' (tratando 'cadastro' como sinônimo) ---

# nome exato de 'Matricula' conforme VR
mat_exato = next((c for c in df_vr.columns if norm_col(c) == "matricula"), "Matricula")

# sinônimos que devemos tratar como 'matricula' no EXTERIOR
SINONIMOS_MATRICULA = {"matricula", "matric", "cadastro", "cad", "registro", "chapa", "idfunc", "idfuncionario", "funcional"}

# 1) mapear colunas do EXTERIOR para nomes exatos da VR
mapping = {}
for c in df_ext.columns:
    nk = norm_col(c)
    if nk in vr_lookup:
        # nome já existe na VR -> renomeia para o exato
        mapping[c] = vr_lookup[nk]
    elif nk in SINONIMOS_MATRICULA:
        # sinônimo de matrícula -> força renome para o nome exato da VR
        mapping[c] = mat_exato

df_ext = df_ext.rename(columns=mapping)

# 2) garantir que a coluna 'Matricula' tenha o nome exato da VR
if mat_exato not in df_ext.columns:
    cand = next((c for c in df_ext.columns if norm_col(c) in SINONIMOS_MATRICULA), None)
    if cand is not None:
        df_ext = df_ext.rename(columns={cand: mat_exato})

# 3) manter somente 'Matricula' (exato conforme VR). Se não existir, cria DF vazio com essa coluna.
if mat_exato in df_ext.columns:
    df_ext = df_ext[[mat_exato]]
else:
    df_ext = pd.DataFrame(columns=[mat_exato])






In [ ]:
# ======================= Tratamento da planilha DESLIGADOS =======================
import os, unicodedata, re
import pandas as pd

BASE_DIR = "/content/dados_vrva"
des_path = os.path.join(BASE_DIR, "DESLIGADOS.xlsx")

# Helpers de normalização/localização de colunas
def _strip_accents_lower(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode("ascii")
    return re.sub(r"\s+", " ", s.strip()).lower()

def _find_col(df: pd.DataFrame, keys) -> str | None:
    """Encontra a melhor coluna cujo nome contenha algum dos tokens de 'keys' (case/acentos-insensitive)."""
    norm_cols = {c: _strip_accents_lower(c) for c in df.columns}
    for c, n in norm_cols.items():
        for k in keys:
            if k in n:
                return c
    return None

def _data_corte(mes_ref: int, ano_ref: int, dia_corte: int) -> pd.Timestamp:
    if mes_ref is None or ano_ref is None or dia_corte is None:
        raise ValueError("Defina MES_REF, ANO_REF e DIA_CORTE antes de tratar DESLIGADOS.")
    base = pd.Timestamp(int(ano_ref), int(mes_ref), 1)
    last_day = (base + pd.offsets.MonthEnd(0)).day
    dia = min(int(dia_corte), int(last_day))
    return pd.Timestamp(int(ano_ref), int(mes_ref), dia)

def _dias_uteis_inclusivo(inicio: pd.Timestamp, fim: pd.Timestamp) -> int:
    if pd.isna(inicio) or pd.isna(fim) or fim < inicio:
        return 0
    return len(pd.bdate_range(inicio, fim))  # dias ÚTEIS, inclusivo

# 1) Ler DESLIGADOS
df_des = pd.read_excel(des_path, engine="openpyxl")

# 2) Localizar colunas-alvo (tolerante a variações)
col_mat = _find_col(df_des, ["matricula", "nr matricula", "n matricula", "numero matricula", "registro"])
col_dt  = _find_col(df_des, ["data demiss", "demissao", "demissão", "rescis", "saida", "saída"])
col_com = _find_col(df_des, ["comunicado deslig", "comunic", "aviso", "desligamento"])

if not col_mat or not col_dt or not col_com:
    raise KeyError(f"Não achei todas as colunas necessárias em DESLIGADOS. "
                   f"Encontradas: {list(df_des.columns)} | "
                   f"matricula={col_mat}, data={col_dt}, comunicado={col_com}")

# 3) Normalizar tipos
df_des[col_dt]  = pd.to_datetime(df_des[col_dt], errors="coerce", dayfirst=True)
df_des[col_com] = df_des[col_com].astype(str).str.strip().str.upper()

# 4) Data de corte (DIA_CORTE/MES_REF/ANO_REF)
DATA_CORTE = _data_corte(globals().get("MES_REF"), globals().get("ANO_REF"), globals().get("DIA_CORTE"))

# 5) Regras de exclusão e cálculo de Dias
mask_excluir = (df_des[col_dt] < DATA_CORTE) & (df_des[col_com] == "OK")
df_des = df_des.loc[~mask_excluir].copy()

# Dias proporcionais para quem tem demissão >= data de corte
df_des["Dias"] = 0
mask_calc = df_des[col_dt] >= DATA_CORTE
df_des.loc[mask_calc, "Dias"] = df_des.loc[mask_calc, col_dt].apply(lambda d: _dias_uteis_inclusivo(DATA_CORTE, d)).astype(int)

# 6) Alinhar nome exato da coluna Matricula com a planilha VR (se df_vr existir)
mat_exato = "Matricula"
if "df_vr" in globals():
    mat_exato = next((c for c in df_vr.columns if _strip_accents_lower(c) == "matricula"), "Matricula")

if col_mat != mat_exato:
    df_des = df_des.rename(columns={col_mat: mat_exato})

# 7) Manter somente Matricula (nome exato) e Dias
df_des = df_des[[mat_exato, "Dias"]].reset_index(drop=True)

# opcional: visualizar amostra
# display(df_des.head())
# ================================================================================


# opcional: ver resultado
df_des.head(100)




,Matricula,Dias
0,33492,0
1,31394,2
2,25670,1
3,34387,1
4,33711,2
5,34819,2
6,35474,2
7,33835,2
8,31880,9
9,34534,11


In [ ]:
# ====================== Colab: Geração de Planilha VR =======================
# Instala dependências (silencioso)
!pip -q install pandas openpyxl xlsxwriter

import os, re, unicodedata
from typing import List, Optional
import pandas as pd
from xlsxwriter.utility import xl_rowcol_to_cell

# --------------------------- Caminhos FIXOS no Colab ------------------------
BASE_DIR = "/content/dados_vrva"
PATH_ATIVOS = f"{BASE_DIR}/ATIVOS.xlsx"
PATH_VR     = f"{BASE_DIR}/VR MENSAL 05.2025.xlsx"
OUTPUT      = f"{BASE_DIR}/NOVA_PLANILHA_VR.xlsx"
AUTO_DOWNLOAD = True  # baixar automaticamente ao final

# Observação: MES_REF e ANO_REF devem existir em alguma célula anterior (ex.: MES_REF=5; ANO_REF=2025).
# Se não existirem, o código tentará extrair do nome do arquivo de VR (padrão MM.AAAA).

# -------------------- Utilidades de normalização e matching -----------------
def _strip_accents_lower(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"\s+", " ", s.strip()).lower()
    return s

def _is_unnamed(val: Optional[str]) -> bool:
    if val is None:
        return True
    s = str(val).strip()
    if s == "" or s.lower() == "nan":
        return True
    return bool(re.match(r"^unnamed[:\s\-]*\d*", s, flags=re.IGNORECASE))

def _detect_header_row(df_raw: pd.DataFrame, scan_rows: int = 15) -> int:
    scan_rows = min(scan_rows, len(df_raw))
    best_idx, best_score = 0, -1
    for i in range(scan_rows):
        row = df_raw.iloc[i]
        score = sum(1 for v in row if not _is_unnamed(v))
        uniques = len(set([str(v).strip().lower() for v in row if not _is_unnamed(v)]))
        dup_penalty = (score - uniques)
        score = score - max(0, dup_penalty // 2)
        if score > best_score:
            best_score = score
            best_idx = i
    return best_idx

def _read_excel_flexible(path: str, sheet_hint: Optional[str] = None) -> pd.DataFrame:
    try:
        df_raw = pd.read_excel(path, sheet_name=sheet_hint if sheet_hint else 0, header=None, dtype=object)
    except ValueError:
        df_raw = pd.read_excel(path, sheet_name=0, header=None, dtype=object)

    header_row = _detect_header_row(df_raw, scan_rows=15)
    header_vals = [str(x).strip() if not pd.isna(x) else "" for x in df_raw.iloc[header_row].tolist()]

    df = df_raw.iloc[header_row+1:].copy()
    df.columns = header_vals

    good_cols, seen = [], set()
    for c in df.columns:
        if _is_unnamed(c):
            continue
        cc = str(c).replace("\n", " ").strip()
        if cc == "":
            continue
        key = cc.lower()
        if key in seen:
            base, k = cc, 2
            while f"{base}.{k}".lower() in seen:
                k += 1
            cc = f"{base}.{k}"
            key = cc.lower()
        seen.add(key)
        good_cols.append(cc)
    df = df.loc[:, good_cols]
    return df

def _best_match_column(target_name: str, candidates: List[str]) -> Optional[str]:
    t_norm = _strip_accents_lower(target_name)
    cand_norm = {c: _strip_accents_lower(c) for c in candidates}

    # exato
    for orig, norm in cand_norm.items():
        if norm == t_norm:
            return orig
    # begins/contains
    for orig, norm in cand_norm.items():
        if norm.startswith(t_norm) or t_norm in norm:
            return orig

    # aliases
    aliases = {
        "matricula": ["matricula", "matrícula", "nr matricula", "n matricula", "numero matricula", "registro"],
        "sindicato do colaborador": [
            "sindicato do colaborador", "sindicato colaborador", "sindicato (colaborador)"
        ],
        "sindicato": ["sindicato", "entidade sindical"],
        "competência": ["competencia", "competência", "comp", "mes/ano", "mes-ano"],
        "valor diário vr": ["valor diário vr", "valor diario vr", "valor vr dia", "vr diário", "vr dia", "valor dia vr"],
        "total": ["total", "valor total", "vr total"],
        "custo empresa": ["custo empresa", "custo da empresa", "custo p/ empresa"],
        "desconto profissional": ["desconto profissional", "desconto do profissional", "desconto colab"],
        "dias": ["dias", "qtd dias", "quantidade dias"],
        "estado": ["estado", "uf", "unidade federativa"],
        "valor": ["valor", "valor (r$)", "valor diario"],
        "admissão": ["admissao", "admissão", "data de admissao", "dt admissao", "dt. admissao"]
    }
    key = t_norm
    if key in aliases:
        for alias in aliases[key]:
            a_norm = _strip_accents_lower(alias)
            for orig, norm in cand_norm.items():
                if norm == a_norm or norm.startswith(a_norm) or a_norm in norm:
                    return orig

    # fallback: maior interseção de tokens
    t_tokens = set(t_norm.split())
    best, best_overlap = None, 0
    for orig, norm in cand_norm.items():
        overlap = len(t_tokens.intersection(set(norm.split())))
        if overlap > best_overlap:
            best, best_overlap = orig, overlap
    return best

def _competencia_str(mes_ref: Optional[int], ano_ref: Optional[int], path_vr: Optional[str]) -> str:
    if mes_ref is not None and ano_ref is not None:
        return f"{int(mes_ref):02d}/{int(ano_ref):04d}"
    if path_vr:
        m = re.search(r"(\d{2})[.\-_/ ]?(\d{4})", os.path.basename(path_vr))
        if m:
            mm, aaaa = m.group(1), m.group(2)
            return f"{mm}/{aaaa}"
    raise ValueError("Defina MES_REF e ANO_REF (ex.: MES_REF=5; ANO_REF=2025) ou renomeie o arquivo VR para conter MM.AAAA.")

def _get_matriculas_set(df: pd.DataFrame) -> set:
    c = _best_match_column("MATRICULA", list(df.columns))
    if not c:
        return set()
    return set(df[c].dropna().astype(str).str.strip())

# ----------------------------- Pipeline principal ---------------------------
def gerar_planilha(path_ativos: str,
                   path_vr: str,
                   path_saida: str,
                   competencia_str: str,
                   sheet_ativos: Optional[str] = "ATIVOS",
                   sheet_vr: Optional[str] = None) -> dict:

    # 1) Lê ATIVOS e VR
    df_ativos = _read_excel_flexible(path_ativos, sheet_hint=sheet_ativos)
    df_vr = _read_excel_flexible(path_vr, sheet_hint=sheet_vr)
    vr_cols = list(df_vr.columns)

    # 1.1) Colunas essenciais em ATIVOS
    c_atv_matricula = _best_match_column("MATRICULA", list(df_ativos.columns))
    c_atv_sindicato = _best_match_column("Sindicato", list(df_ativos.columns))
    if not c_atv_matricula:
        raise KeyError(f"Não encontrei a coluna de Matrícula no ATIVOS. Colunas: {list(df_ativos.columns)}")

    # 1.2) EXCLUSÕES de matrículas (df_afa, df_apr, df_des, df_est, df_ext, df_fer)
    excl_names = ["df_afa","df_apr","df_des","df_est","df_ext","df_fer"]
    excl_set = set()
    for nm in excl_names:
        if nm in globals() and isinstance(globals()[nm], pd.DataFrame):
            excl_set |= _get_matriculas_set(globals()[nm])

    before_excl = len(df_ativos)
    if excl_set:
        df_ativos = df_ativos[~df_ativos[c_atv_matricula].astype(str).str.strip().isin(excl_set)].copy()
    removed_count = before_excl - len(df_ativos)

    # 1.3) INCLUSÕES de df_adm + captura de Admissão/Sindicato/Estado
    adm_included = 0
    adm_map = {}        # {matricula -> datetime(Admissão)}
    adm_estado_map = {} # {matricula -> ESTADO (upper/normalized)}
    adm_sind_map = {}   # {matricula -> sindicato (texto)}

    if "df_adm" in globals() and isinstance(globals()["df_adm"], pd.DataFrame):
        df_adm = globals()["df_adm"]
        c_adm_matricula = _best_match_column("MATRICULA", list(df_adm.columns))
        c_adm_adm       = _best_match_column("Admissão", list(df_adm.columns))
        c_adm_sind      = _best_match_column("Sindicato", list(df_adm.columns))
        c_adm_estado    = _best_match_column("ESTADO", list(df_adm.columns))

        if c_adm_matricula:
            mats_adm_all = df_adm[c_adm_matricula].dropna().astype(str).str.strip()
            mats_ativos = set(df_ativos[c_atv_matricula].astype(str).str.strip())
            mats_to_add = [m for m in mats_adm_all if m not in mats_ativos]

            # dicionários auxiliares
            if c_adm_adm:
                for _, r in df_adm[[c_adm_matricula, c_adm_adm]].dropna(subset=[c_adm_matricula]).iterrows():
                    m = str(r[c_adm_matricula]).strip()
                    dt = pd.to_datetime(r[c_adm_adm], errors="coerce")
                    if pd.notna(dt):
                        adm_map[m] = dt

            if c_adm_sind:
                for _, r in df_adm[[c_adm_matricula, c_adm_sind]].dropna(subset=[c_adm_matricula]).iterrows():
                    adm_sind_map[str(r[c_adm_matricula]).strip()] = r[c_adm_sind]

            if c_adm_estado:
                def _norm_upper(s): return _strip_accents_lower(str(s)).upper()
                for _, r in df_adm[[c_adm_matricula, c_adm_estado]].dropna(subset=[c_adm_matricula]).iterrows():
                    adm_estado_map[str(r[c_adm_matricula]).strip()] = _norm_upper(r[c_adm_estado])

            if mats_to_add:
                # prepara df_extra com mesmas colunas de ATIVOS
                df_extra = pd.DataFrame(columns=df_ativos.columns)
                df_extra = df_extra.reindex(range(len(mats_to_add)))
                df_extra[c_atv_matricula] = mats_to_add

                # se ATIVOS tem coluna de sindicato e df_adm também, replica para ajudar o cálculo de VR
                if c_atv_sindicato and c_atv_sindicato in df_extra.columns and c_adm_sind:
                    df_extra[c_atv_sindicato] = [adm_sind_map.get(m) for m in mats_to_add]

                # concatena
                df_ativos = pd.concat([df_ativos, df_extra], ignore_index=True)
                adm_included = len(mats_to_add)

    # 2) Base nova com as mesmas colunas da VR (ATIVOS já filtrado/expandido)
    df_new = pd.DataFrame(columns=vr_cols, index=range(len(df_ativos)))

    # 3) Mapeamentos de colunas para a base final
    c_new_matricula = _best_match_column("Matricula", vr_cols)
    c_new_sindcolab = _best_match_column("Sindicato do Colaborador", vr_cols)
    c_new_compet    = _best_match_column("Competência", vr_cols)
    c_new_admissao  = _best_match_column("Admissão", vr_cols)

    # 4) Copia dados-base
    if c_atv_matricula and c_new_matricula:
        df_new[c_new_matricula] = df_ativos[c_atv_matricula].values
    if c_atv_sindicato and c_new_sindcolab:
        df_new[c_new_sindcolab] = df_ativos[c_atv_sindicato].values

    # 5) Competência
    if c_new_compet and c_new_compet in df_new.columns:
        df_new[c_new_compet] = competencia_str
    else:
        df_new["Competência"] = competencia_str
        c_new_compet = "Competência"

    # 5.1) Admissão — padroniza como datetime sem hora (DD/MM/AAAA)
    dest_adm = c_new_admissao if c_new_admissao in df_new.columns else "Admissão"
    mats_series = df_new[c_new_matricula].astype(str).str.strip() if c_new_matricula in df_new.columns else pd.Series(index=df_new.index, dtype="object")

    # valores vindos do df_adm (se houver)
    df_new[dest_adm] = mats_series.map(adm_map) if adm_map else pd.NaT

    # Converte para datetime, entendendo dia primeiro (pt-BR) e removendo hora/timezone
    df_new[dest_adm] = pd.to_datetime(df_new[dest_adm], errors="coerce", dayfirst=True)
    try:
        df_new[dest_adm] = df_new[dest_adm].dt.tz_localize(None)
    except Exception:
        pass
    df_new[dest_adm] = df_new[dest_adm].dt.normalize()  # zera hora para 00:00:00

    # 6) Dias = 22 (default) + override pelas matrículas de df_des
    c_new_dias = _best_match_column("Dias", vr_cols)
    destino_dias = c_new_dias if c_new_dias in df_new.columns else "Dias"
    df_new[destino_dias] = 22  # default

    # Se existir df_des com colunas Matricula e Dias, sobrescreve
    if "df_des" in globals() and isinstance(df_des, pd.DataFrame) and not df_des.empty:
        c_des_mat  = _best_match_column("Matricula", list(df_des.columns))
        c_des_dias = _best_match_column("Dias",      list(df_des.columns))
        if c_des_mat and c_des_dias and c_new_matricula in df_new.columns:
            # mapa matricula -> dias (normalizando chaves e garantindo número)
            des_map = (
                df_des[[c_des_mat, c_des_dias]]
                .dropna(subset=[c_des_mat])
                .assign(_mat=lambda d: d[c_des_mat].astype(str).str.strip(),
                        _dias=lambda d: d[c_des_dias].apply(lambda v: str(v).replace(",", ".")))
                .set_index("_mat")["_dias"]
            )
            des_map = pd.to_numeric(des_map, errors="coerce")  # converte para número

            # aplica às linhas correspondentes
            keys = df_new[c_new_matricula].astype(str).str.strip()
            override = keys.map(des_map)  # Series alinhada por índice de df_new
            mask = override.notna()
            df_new.loc[mask, destino_dias] = override[mask].astype(float)

    # 7) VALOR DIÁRIO VR via planilha referência (Estado -> Valor) + DE-PARA
    c_new_valor_dia = _best_match_column("VALOR DIÁRIO VR", vr_cols)
    destino_valor = c_new_valor_dia if c_new_valor_dia in df_new.columns else "VALOR DIÁRIO VR"

    path_ref = os.path.join(BASE_DIR, "Base sindicato x valor.xlsx")
    if not os.path.exists(path_ref):
        raise FileNotFoundError(f"Não achei a planilha de referência: {path_ref}")
    df_ref = _read_excel_flexible(path_ref, sheet_hint=None)

    c_ref_estado = _best_match_column("ESTADO", list(df_ref.columns))
    c_ref_valor  = _best_match_column("VALOR",  list(df_ref.columns))
    if not c_ref_estado or not c_ref_valor:
        raise KeyError(
            "Não encontrei as colunas 'ESTADO' e/ou 'VALOR' na planilha de referência. "
            f"Colunas encontradas: {list(df_ref.columns)}"
        )

    def _norm_upper(s): return _strip_accents_lower(s).upper()
    ref_map = {}
    for _, r in df_ref[[c_ref_estado, c_ref_valor]].dropna(subset=[c_ref_estado]).iterrows():
        estado_norm = _norm_upper(str(r[c_ref_estado]))
        try:
            valor_num = float(str(r[c_ref_valor]).replace(",", "."))
        except Exception:
            valor_num = pd.to_numeric(r[c_ref_valor], errors="coerce")
            valor_num = float(valor_num) if pd.notna(valor_num) else 0.0
        ref_map[estado_norm] = valor_num

    sindicato_to_estado = {
        "SINDPD SP": "SÃO PAULO",
        "SIND.TRAB.EM PROC DADOS E EMPR.EMPRESAS PROC DADOS ESTADO DE SP.": "SÃO PAULO",
        "SINDPPD RS": "RIO GRANDE DO SUL",
        "SINDICATO DOS TRAB. EM PROC. DE DADOS RIO GRANDE DO SUL": "RIO GRANDE DO SUL",
        "SINDPD RJ": "RIO DE JANEIRO",
        "SINDICATO PROFISSIONAIS DE PROC DADOS DO RIO DE JANEIRO": "RIO DE JANEIRO",
        "SITEPD PR": "PARANÁ",
        "SIND DOS TRAB EM EMPR PRIVADAS DE PROC DE DADOS DE CURITIBA E REGIAO METROPOLITANA": "PARANÁ"
    }

    vr_val = pd.Series(0.0, index=df_new.index, dtype="float64")
    sind_series = df_new[c_new_sindcolab].astype(str).fillna("") if c_new_sindcolab in df_new.columns else pd.Series("", index=df_new.index)

    for i in df_new.index:
        s_norm = _norm_upper(sind_series.iloc[i]) if len(sind_series) else ""
        estado = None
        # 1) tenta mapear pelo Sindicato
        for key, est in sindicato_to_estado.items():
            if s_norm.startswith(_norm_upper(key)):
                estado = est; break
        if not estado:
            for key, est in sindicato_to_estado.items():
                if _norm_upper(key) in s_norm:
                    estado = est; break
        # 2) fallback: usa ESTADO do df_adm (se houver) para essa matrícula
        if not estado and c_new_matricula in df_new.columns:
            mat = str(df_new.at[i, c_new_matricula]).strip()
            if mat in adm_estado_map:
                estado = adm_estado_map[mat]

        # aplica valor (0.0 se não encontrar)
        if estado and _norm_upper(estado) in ref_map:
            vr_val.at[i] = ref_map[_norm_upper(estado)]
        else:
            vr_val.at[i] = 0.0

    df_new[destino_valor] = vr_val

    # 8) TOTAL = Dias * VALOR DIÁRIO VR
    c_new_total     = _best_match_column("TOTAL", vr_cols)
    destino_total = c_new_total if c_new_total in df_new.columns else "TOTAL"
    dias_num   = pd.to_numeric(df_new[destino_dias], errors="coerce").fillna(0)
    valor_num  = pd.to_numeric(df_new[destino_valor], errors="coerce").fillna(0.0)
    df_new[destino_total] = (dias_num * valor_num).round(2)

    # 9) Custo empresa = 80%
    c_new_custo     = _best_match_column("Custo empresa", vr_cols)
    destino_custo = c_new_custo if c_new_custo in df_new.columns else "Custo empresa"
    df_new[destino_custo] = (df_new[destino_total] * 0.80).round(2)

    # 10) Desconto profissional = TOTAL - Custo empresa
    c_new_desc      = _best_match_column("Desconto profissional", vr_cols)
    destino_desc  = c_new_desc if c_new_desc in df_new.columns else "Desconto profissional"
    df_new[destino_desc] = (df_new[destino_total] - df_new[destino_custo]).round(2)

    # 11) Exporta com cabeçalho na segunda linha + formatação + soma topo
    with pd.ExcelWriter(path_saida, engine="xlsxwriter") as writer:
        # força dtype numérico nas colunas calculadas
        for col_name in [destino_dias, destino_valor, destino_total, destino_custo, destino_desc]:
            if col_name in df_new.columns:
                df_new[col_name] = pd.to_numeric(df_new[col_name], errors="coerce")

        # garante datetime na coluna Admissão para formatação no Excel
        if dest_adm in df_new.columns:
            df_new[dest_adm] = pd.to_datetime(df_new[dest_adm], errors="coerce")

        df_new.to_excel(writer, index=False, sheet_name="BASE", startrow=1)
        ws = writer.sheets["BASE"]

        # cálculo automático
        writer.book.set_calc_mode('automatic')
        ws.freeze_panes(2, 0)

        # formatos
        num_fmt = writer.book.add_format({'num_format': '#,##0.00'})
        date_fmt = writer.book.add_format({'num_format': 'dd/mm/yyyy'})

        # aplica formato às colunas numéricas
        for col_name in [destino_dias, destino_valor, destino_total, destino_custo, destino_desc]:
            if col_name in df_new.columns:
                col_idx = df_new.columns.get_loc(col_name)
                ws.set_column(col_idx, col_idx, 15, num_fmt)

        # aplica formato de data (DD/MM/AAAA) à coluna Admissão
        if dest_adm in df_new.columns:
            col_idx = df_new.columns.get_loc(dest_adm)
            ws.set_column(col_idx, col_idx, 12, date_fmt)

        # somatório no topo do TOTAL (linha 1)
        if destino_total in df_new.columns:
            total_col_idx = df_new.columns.get_loc(destino_total)
            first_data_row = 3
            last_data_row = len(df_new) + 2
            top_cell = xl_rowcol_to_cell(0, total_col_idx)
            range_str = f"{xl_rowcol_to_cell(first_data_row-1, total_col_idx)}:{xl_rowcol_to_cell(last_data_row-1, total_col_idx)}"
            soma_val = float(df_new[destino_total].sum().round(2))
            ws.write_formula(top_cell, f"=SUM({range_str})", num_fmt, soma_val)

    return {
        "linhas_saida": len(df_new),
        "colunas_saida": list(df_new.columns),
        "arquivo": path_saida,
        "mapeamentos": {
            "Competência": competencia_str,
            "Excluídas por status": int(removed_count),
            "Incluídas de df_adm": int(adm_included),
            "Coluna Admissão destino": dest_adm,
            "Dias": destino_dias,
            "VALOR DIÁRIO VR": destino_valor,
            "TOTAL": destino_total,
            "Custo empresa": destino_custo,
            "Desconto profissional": destino_desc,
        }
    }

# ------------------------------ Execução prática ----------------------------
if not os.path.exists(PATH_ATIVOS):
    raise FileNotFoundError(f"Arquivo ATIVOS não encontrado em: {PATH_ATIVOS}")
if not os.path.exists(PATH_VR):
    raise FileNotFoundError(f"Arquivo de VR não encontrado em: {PATH_VR}")

_comp_str = _competencia_str(
    mes_ref=globals().get("MES_REF"),
    ano_ref=globals().get("ANO_REF"),
    path_vr=PATH_VR
)

print(">>> Arquivo ATIVOS:", PATH_ATIVOS)
print(">>> Arquivo VR    :", PATH_VR)
print(">>> Competência   :", _comp_str)

info = gerar_planilha(
    PATH_ATIVOS,
    PATH_VR,
    OUTPUT,
    competencia_str=_comp_str,
    sheet_ativos="ATIVOS",
    sheet_vr=None
)

print("\n✅ Arquivo gerado:")
print("  - Caminho:", info["arquivo"])
print("  - Linhas :", info["linhas_saida"])
print("  - Colunas:", len(info["colunas_saida"]))
print("  - Mapeamentos:")
for k, v in info["mapeamentos"].items():
    print(f"    • {k}: {v}")

if AUTO_DOWNLOAD:
    try:
        from google.colab import files
        files.download(OUTPUT)
    except Exception as e:
        print("[AVISO] Não foi possível iniciar o download automático:", e)
# ============================================================================


>>> Arquivo ATIVOS: /content/dados_vrva/ATIVOS.xlsx
>>> Arquivo VR    : /content/dados_vrva/VR MENSAL 05.2025.xlsx
>>> Competência   : 05/2025

✅ Arquivo gerado:
  - Caminho: /content/dados_vrva/NOVA_PLANILHA_VR.xlsx
  - Linhas : 1796
  - Colunas: 10
  - Mapeamentos:
    • Competência: 05/2025
    • Excluídas por status: 23
    • Incluídas de df_adm: 4
    • Coluna Admissão destino: Admissão
    • Dias: Dias
    • VALOR DIÁRIO VR: VALOR DIÁRIO VR
    • TOTAL: TOTAL
    • Custo empresa: Custo empresa
    • Desconto profissional: Desconto profissional


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
"""
Mediador CCT/ACT Scraper (MTE)
--------------------------------
Objetivo: Dado um CNPJ (e, opcionalmente, o nome do sindicato e uma data de referência),
abrir o Sistema Mediador do MTE, consultar os instrumentos coletivos (CCT/ACT),
listar resultados, selecionar o instrumento aplicável pela vigência e baixar o PDF.
Depois, extrair regras de jornada/escala e banco de horas do PDF em texto estruturado.


Requisitos (instale no ambiente antes de rodar):
    pip install selenium undetected-chromedriver webdriver-manager pdfminer.six beautifulsoup4 pandas python-dateutil

Uso básico (exemplo):
    from mediador_cct_act_scraper import MediadorScraper

    scraper = MediadorScraper(download_dir="./ccts", headless=True)
    resultados = scraper.buscar_por_cnpj("12.345.678/0001-90")
    # Opcional: ver todos os achados
    for r in resultados:
        print(r)

    # Selecionar o instrumento aplicável para julho/2025, filtrando por sindicato (se quiser)
    escolhido = MediadorScraper.selecionar_aplicavel(resultados, mes=7, ano=2025, sindicato_nome="SINDICATO X")
    if escolhido:
        pdf_path = scraper.baixar_pdf(escolhido)
        regras = scraper.extrair_regras_pdf(pdf_path)
        print(regras)

    scraper.fechar()


"""
from __future__ import annotations

import os
import re
import time
import json
import shutil
import zipfile
from dataclasses import dataclass, asdict
from typing import List, Optional, Dict, Any

from datetime import datetime
from dateutil import parser as dateparser
from dateutil.relativedelta import relativedelta

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from bs4 import BeautifulSoup
import pandas as pd

from pdfminer.high_level import extract_text

# -------------------------------
# Utilidades
# -------------------------------

def normalizar_cnpj(cnpj: str) -> str:
    """Remove caracteres não numéricos do CNPJ."""
    return re.sub(r"\D", "", cnpj or "")


def parse_data_br(texto: str) -> Optional[datetime]:
    """Tenta parsear uma data BR (dd/mm/aaaa)."""
    if not texto:
        return None
    m = re.search(r"(\d{1,2})/(\d{1,2})/(\d{4})", texto)
    if not m:
        return None
    d, mth, y = map(int, m.groups())
    try:
        return datetime(y, mth, d)
    except Exception:
        return None


@dataclass
class Instrumento:
    tipo: str  # CCT/ACT/TA
    numero_registro: Optional[str]
    numero_processo: Optional[str]
    vigencia_inicio: Optional[datetime]
    vigencia_fim: Optional[datetime]
    sindicato_laboral: Optional[str]
    sindicato_patronal: Optional[str]
    cnpj_empresa: Optional[str]
    municipio: Optional[str]
    uf: Optional[str]
    link_detalhe: Optional[str]
    link_pdf: Optional[str] = None
    titulo: Optional[str] = None

    def cobre_mes_ano(self, mes: int, ano: int) -> bool:
        if not self.vigencia_inicio or not self.vigencia_fim:
            return False
        base = datetime(ano, mes, 15)
        return self.vigencia_inicio <= base <= self.vigencia_fim


class MediadorScraper:
    BASE_URL = "https://www3.mte.gov.br/sistemas/mediador/"
    CONSULTA_URL = BASE_URL + "consultarinstcoletivo"

    def __init__(self, download_dir: str = "./downloads_mediador", headless: bool = True, timeout: int = 25):
        self.download_dir = os.path.abspath(download_dir)
        os.makedirs(self.download_dir, exist_ok=True)
        self.timeout = timeout
        self.driver = self._init_driver(headless=headless)

    def _init_driver(self, headless: bool = True):
        opts = uc.ChromeOptions()
        if headless:
            opts.add_argument("--headless=new")
        opts.add_argument("--no-sandbox")
        opts.add_argument("--disable-dev-shm-usage")
        prefs = {
            "download.default_directory": self.download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True,
        }
        opts.add_experimental_option("prefs", prefs)
        driver = uc.Chrome(options=opts)
        driver.set_page_load_timeout(self.timeout)
        return driver

    def fechar(self):
        try:
            self.driver.quit()
        except Exception:
            pass

    # -------------------------------
    # Consulta por CNPJ
    # -------------------------------
    def buscar_por_cnpj(self, cnpj: str) -> List[Instrumento]:
        cnpj_digits = normalizar_cnpj(cnpj)
        if len(cnpj_digits) != 14:
            raise ValueError("CNPJ inválido. Informe 14 dígitos (com ou sem pontuação).")

        self.driver.get(self.CONSULTA_URL)
        # Espera por possível captcha
        self._aguardar_captcha(max_espera=60)

        # Tenta localizar o campo CNPJ de forma robusta (por label, por placeholder, por proximidade de texto)
        campo_cnpj = self._encontrar_input_relacionado_a_texto(["CNPJ", "CNPJ do participante", "CNPJ:"])
        if not campo_cnpj:
            raise RuntimeError("Não foi possível localizar o campo CNPJ na página de consulta.")
        campo_cnpj.clear()
        campo_cnpj.send_keys(cnpj_digits)

        # Tenta clicar no botão de pesquisar
        botao = self._encontrar_botao_por_texto(["Pesquisar", "Consultar", "Buscar", "Localizar"])
        if not botao:
            raise RuntimeError("Não foi possível localizar o botão de pesquisa na página de consulta.")
        botao.click()

        # Aguarda resultados renderizarem
        time.sleep(2)
        self._aguardar_resultados()

        html = self.driver.page_source
        return self._parse_resultados(html, cnpj_digits)

    # -------------------------------
    # Consulta por sindicato (fallback)
    # -------------------------------
    def buscar_por_sindicato(self, nome: str, uf: Optional[str] = None, municipio: Optional[str] = None) -> List[Instrumento]:
        self.driver.get(self.CONSULTA_URL)
        self._aguardar_captcha(max_espera=60)

        # Campo "Participante" / "Sindicatos" costuma existir
        campo_participante = self._encontrar_input_relacionado_a_texto(["Participante", "Sindic", "Entidade", "Razão Social"])
        if not campo_participante:
            raise RuntimeError("Não foi possível localizar o campo de participante/sindicato.")
        campo_participante.clear()
        campo_participante.send_keys(nome)

        # UF/município são opcionais
        if uf:
            campo_uf = self._encontrar_input_relacionado_a_texto(["UF", "Estado"])
            if campo_uf:
                try:
                    campo_uf.clear(); campo_uf.send_keys(uf)
                except Exception:
                    pass
        if municipio:
            campo_mun = self._encontrar_input_relacionado_a_texto(["Município", "Municipio"])
            if campo_mun:
                try:
                    campo_mun.clear(); campo_mun.send_keys(municipio)
                except Exception:
                    pass

        botao = self._encontrar_botao_por_texto(["Pesquisar", "Consultar", "Buscar", "Localizar"])
        if not botao:
            raise RuntimeError("Não foi possível localizar o botão de pesquisa na página de consulta.")
        botao.click()

        time.sleep(2)
        self._aguardar_resultados()
        html = self.driver.page_source
        return self._parse_resultados(html, None)

    # -------------------------------
    # Parsing de resultados
    # -------------------------------
    def _parse_resultados(self, html: str, cnpj_digits: Optional[str]) -> List[Instrumento]:
        soup = BeautifulSoup(html, "html.parser")
        instrumentos = []

        # Procura a primeira tabela "grande"
        tabelas = soup.find_all("table")
        for tab in tabelas:
            linhas = tab.find_all("tr")
            # Heurística: tabela de resultados costuma ter cabeçalho com "Vigência" ou "Registro"
            header = [th.get_text(strip=True) for th in tab.find_all("th")]
            if not header:
                # Algumas versões usam <td> como cabeçalho
                header = [td.get_text(strip=True) for td in linhas[0].find_all("td")] if linhas else []
            cabecalho_ok = any("Vig" in h or "Registro" in h or "Instrumento" in h for h in header)
            if not cabecalho_ok:
                continue

            # Processa linhas
            for tr in linhas[1:]:
                cols = [c.get_text(" ", strip=True) for c in tr.find_all(["td", "th"])]
                if len(cols) < 3:
                    continue
                link_det = tr.find("a")
                href = link_det.get("href") if link_det else None
                # Extrai campos por heurística
                tipo = ""
                numero_registro = None
                numero_processo = None
                vig_ini = vig_fim = None
                sindicato_l = sindicato_p = None
                municipio = uf = None
                titulo = None

                row_text = " ".join(cols)
                titulo = row_text[:180]

                # Tipo do instrumento (CCT/ACT/TA)
                m_tipo = re.search(r"\b(Conven\w+ Coletiv\w+|Acordo Coletiv\w+|Termo Aditiv\w+)\b", row_text, re.I)
                if m_tipo:
                    palavra = m_tipo.group(1).lower()
                    if "acordo" in palavra:
                        tipo = "ACT"
                    elif "termo" in palavra:
                        tipo = "TA"
                    else:
                        tipo = "CCT"

                # Número de registro
                m_reg = re.search(r"Registro\s*(?:no\s*MTE|MTE)?\s*[:\-]?\s*([A-Z]{2}\d{6}/\d{4}|\w{2,}\d{2,}/\d{4})", row_text, re.I)
                if m_reg:
                    numero_registro = m_reg.group(1)

                # Número do processo
                m_proc = re.search(r"Processo\s*[:\-]?\s*([\d\./\-]+)", row_text, re.I)
                if m_proc:
                    numero_processo = m_proc.group(1)

                # Vigência (dd/mm/aaaa a dd/mm/aaaa)
                m_vig = re.search(r"(\d{2}/\d{2}/\d{4}).*?(\d{2}/\d{2}/\d{4})", row_text)
                if m_vig:
                    vig_ini = parse_data_br(m_vig.group(1))
                    vig_fim = parse_data_br(m_vig.group(2))

                # Sindicatos
                m_sind_l = re.search(r"Sindic\w+\s+Laboral\s*[:\-]?\s*(.+?)\s{2,}", row_text, re.I)
                m_sind_p = re.search(r"Sindic\w+\s+Patronal\s*[:\-]?\s*(.+?)\s{2,}", row_text, re.I)
                if m_sind_l:
                    sindicato_l = m_sind_l.group(1).strip()
                if m_sind_p:
                    sindicato_p = m_sind_p.group(1).strip()

                # UF/município
                m_uf = re.search(r"\b(UF|Estado)[:\-]?\s*([A-Z]{2})\b", row_text)
                if m_uf:
                    uf = m_uf.group(2)
                m_mun = re.search(r"Munic[ií]pio[:\-]?\s*([A-Za-z \-']{3,})\b", row_text)
                if m_mun:
                    municipio = m_mun.group(1).strip()

                instrumentos.append(Instrumento(
                    tipo=tipo or "",
                    numero_registro=numero_registro,
                    numero_processo=numero_processo,
                    vigencia_inicio=vig_ini,
                    vigencia_fim=vig_fim,
                    sindicato_laboral=sindicato_l,
                    sindicato_patronal=sindicato_p,
                    cnpj_empresa=cnpj_digits,
                    municipio=municipio,
                    uf=uf,
                    link_detalhe=(href if href and href.startswith("http") else (self.BASE_URL + href.lstrip("/")) if href else None),
                    titulo=titulo,
                ))

        # Dedup básica por (registro, link)
        uniq = {}
        for it in instrumentos:
            k = (it.numero_registro, it.link_detalhe)
            if k not in uniq:
                uniq[k] = it
        return list(uniq.values())

    def _aguardar_resultados(self):
        # Qualquer tabela ou elemento com "Resultados"/"Instrumento"/"Vigência"
        try:
            WebDriverWait(self.driver, self.timeout).until(
                EC.presence_of_element_located((By.XPATH, "//*[contains(translate(text(),'VIG','vig'),'vig')]"))
            )
        except TimeoutException:
            pass

    # -------------------------------
    # Seleção do instrumento aplicável
    # -------------------------------
    @staticmethod
    def selecionar_aplicavel(resultados: List[Instrumento], mes: int, ano: int, sindicato_nome: Optional[str] = None, preferir_tipo: Optional[str] = None) -> Optional[Instrumento]:
        """Escolhe o instrumento cuja vigência cobre (mes/ano). Se sindicato_nome for dado,
        filtra pelo nome aproximado. preferir_tipo pode ser "CCT" ou "ACT"."""
        def _norm(s):
            return re.sub(r"[^a-z0-9]", "", s.lower()) if s else ""

        alvo = datetime(ano, mes, 15)
        cand = []
        for it in resultados:
            if it.vigencia_inicio and it.vigencia_fim and it.vigencia_inicio <= alvo <= it.vigencia_fim:
                if sindicato_nome:
                    n = _norm(sindicato_nome)
                    if n not in _norm(it.sindicato_laboral) and n not in _norm(it.titulo or ""):
                        continue
                cand.append(it)

        if not cand:
            return None

        # Preferir tipo, depois mais recente pela data de início
        if preferir_tipo:
            preferir_tipo = preferir_tipo.upper()
            cand_tipo = [c for c in cand if c.tipo.upper() == preferir_tipo]
            if cand_tipo:
                cand = cand_tipo

        cand.sort(key=lambda x: (x.vigencia_inicio or datetime.min), reverse=True)
        return cand[0]

    # -------------------------------
    # Baixar PDF
    # -------------------------------
    def baixar_pdf(self, instrumento: Instrumento) -> str:
        """Abre a página de detalhe e tenta achar um link de PDF para baixar."""
        if not instrumento.link_detalhe:
            raise ValueError("Instrumento não possui link de detalhe.")

        self.driver.get(instrumento.link_detalhe)
        self._aguardar_captcha(max_espera=60)
        time.sleep(2)

        # Procura links para PDF
        links = self.driver.find_elements(By.XPATH, "//a[contains(@href,'.pdf') or contains(translate(text(),'PDF','pdf'),'pdf')]")
        href = None
        for a in links:
            try:
                h = a.get_attribute("href")
                if h and h.lower().endswith(".pdf"):
                    href = h
                    break
            except Exception:
                pass

        if not href:
            # Tenta capturar PDF via botão de imprimir/visualizar
            possiveis = self.driver.find_elements(By.XPATH, "//a | //button")
            for el in possiveis:
                try:
                    txt = (el.text or "").strip().lower()
                    if any(p in txt for p in ["pdf", "imprimir", "visualizar", "ver inteiro", "abrir"]):
                        el.click()
                        time.sleep(2)
                        # após click, checar novamente se há <a href=...pdf>
                        links2 = self.driver.find_elements(By.XPATH, "//a[contains(@href,'.pdf')]")
                        if links2:
                            href = links2[0].get_attribute("href")
                            break
                except Exception:
                    continue

        if not href:
            raise RuntimeError("Não foi possível localizar o link do PDF no detalhe do instrumento.")

        # Download manual (usando requests pode falhar por sessão; então usar o próprio navegador)
        # Vamos abrir o PDF em nova aba e salvar via streaming do Chrome (diretório de downloads)
        self.driver.execute_script("window.open(arguments[0], '_blank');", href)
        time.sleep(3)

        # Como alternativa simples, baixar via urllib com cookies
        pdf_name = self._nome_pdf(instrumento, href)
        pdf_path = os.path.join(self.download_dir, pdf_name)

        # Fallback: tentar baixar com requests + cookies do Selenium
        try:
            import requests
            s = requests.Session()
            for c in self.driver.get_cookies():
                s.cookies.set(c['name'], c['value'], domain=c.get('domain'))
            r = s.get(href, timeout=30)
            r.raise_for_status()
            with open(pdf_path, 'wb') as f:
                f.write(r.content)
        except Exception:
            # Último recurso: o arquivo pode estar na pasta de download do Chrome headless
            # Tentativa de achar algum .pdf recém criado lá
            time.sleep(5)
            candidatos = [f for f in os.listdir(self.download_dir) if f.lower().endswith('.pdf')]
            if candidatos:
                # pega o mais recente
                cand = max([os.path.join(self.download_dir, f) for f in candidatos], key=os.path.getmtime)
                shutil.move(cand, pdf_path)
            else:
                raise

        instrumento.link_pdf = href
        return pdf_path

    def _nome_pdf(self, it: Instrumento, href: str) -> str:
        base = it.numero_registro or it.titulo or os.path.basename(href).split('?')[0]
        base = re.sub(r"\W+", "_", base)[:80]
        return f"{base}.pdf"

    # -------------------------------
    # Extração de regras do PDF
    # -------------------------------
    @staticmethod
    def extrair_regras_pdf(pdf_path: str) -> Dict[str, Any]:
        """Extrai texto do PDF e busca padrões comuns de jornada/escala/banco de horas.
        Retorna um dicionário com achados e trechos relevantes.
        """
        texto = extract_text(pdf_path) or ""
        texto_norm = re.sub(r"\s+", " ", texto)

        # Heurísticas de extração
        regras: Dict[str, Any] = {
            "jornada_semanais_horas": None,
            "jornada_mensais_horas": None,
            "escala": None,
            "banco_de_horas": None,
            "intervalo_intrajornada": None,
            "repouso_semanal": None,
            "horas_extras": None,
            "trechos_relevantes": {}
        }

        # Jornada semanal/mensal
        m_sem = re.search(r"(\b3[0-9]|4[0-8])\s*horas\s*semanais", texto_norm, re.I)
        if m_sem:
            regras["jornada_semanais_horas"] = int(re.sub("\D", "", m_sem.group(0))[:2])
            regras["trechos_relevantes"]["jornada_semanais"] = m_sem.group(0)
        m_mens = re.search(r"(1\d\d|2\d\d)\s*horas\s*mensais", texto_norm, re.I)
        if m_mens:
            regras["jornada_mensais_horas"] = int(re.sub("\D", "", m_mens.group(0))[:3])
            regras["trechos_relevantes"]["jornada_mensais"] = m_mens.group(0)

        # Escalas típicas
        m_esc = re.search(r"\b(12\s*[xX]\s*36|6\s*[xX]\s*1|5\s*[xX]\s*2|24\s*[xX]\s*72|4\s*[xX]\s*2)\b", texto_norm)
        if m_esc:
            regras["escala"] = re.sub(r"\s+", "", m_esc.group(1))
            regras["trechos_relevantes"]["escala"] = m_esc.group(0)

        # Banco de horas / compensação
        m_bh = re.search(r"(banco\s+de\s+horas|compensa[cç][aã]o\s+de\s+jornada).*?(\.|;)", texto_norm, re.I)
        if m_bh:
            regras["banco_de_horas"] = m_bh.group(0)
            regras["trechos_relevantes"]["banco_de_horas"] = m_bh.group(0)

        # Intervalo intrajornada
        m_int = re.search(r"intervalo\s+intrajornada.*?(\.|;)", texto_norm, re.I)
        if m_int:
            regras["intervalo_intrajornada"] = m_int.group(0)
            regras["trechos_relevantes"]["intervalo_intrajornada"] = m_int.group(0)

        # Repouso semanal
        m_rsr = re.search(r"repouso\s+semanal\s+remunerado.*?(\.|;)", texto_norm, re.I)
        if m_rsr:
            regras["repouso_semanal"] = m_rsr.group(0)
            regras["trechos_relevantes"]["rsr"] = m_rsr.group(0)

        # Horas extras
        m_he = re.search(r"hora[s]?\s+extra[s]?.*?(\.|;)", texto_norm, re.I)
        if m_he:
            regras["horas_extras"] = m_he.group(0)
            regras["trechos_relevantes"]["horas_extras"] = m_he.group(0)

        regras["_debug_tamanho_texto"] = len(texto_norm)
        return regras

    # -------------------------------
    # Helpers de página
    # -------------------------------
    def _aguardar_captcha(self, max_espera: int = 60):
        """Se aparecer "captcha" no HTML, aguarda até sumir (ou até max_espera)."""
        t0 = time.time()
        while time.time() - t0 < max_espera:
            html = (self.driver.page_source or "").lower()
            if "captcha" in html and "cloudflare" not in html:
                print("[Aviso] Captcha detectado: resolva manualmente na aba do navegador...")
                time.sleep(5)
            else:
                return

    def _encontrar_input_relacionado_a_texto(self, textos: List[str]):
        # Busca por label + input
        for t in textos:
            try:
                el = self.driver.find_element(By.XPATH, f"//label[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'), '{t.lower()}')]/following::input[1]")
                if el:
                    return el
            except Exception:
                pass
        # Busca por texto solto seguido de input
        for t in textos:
            try:
                el = self.driver.find_element(By.XPATH, f"//*[contains(translate(normalize-space(text()),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'), '{t.lower()}')]/following::input[1]")
                if el:
                    return el
            except Exception:
                pass
        # Fallback: primeiro input da página
        try:
            return self.driver.find_element(By.XPATH, "//input[@type='text' or @type='search']")
        except Exception:
            return None

    def _encontrar_botao_por_texto(self, textos: List[str]):
        for t in textos:
            try:
                el = self.driver.find_element(By.XPATH, f"//button[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'), '{t.lower()}')]")
                if el: return el
            except Exception:
                pass
        for t in textos:
            try:
                el = self.driver.find_element(By.XPATH, f"//input[@type='submit' and contains(translate(@value,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'), '{t.lower()}')]")
                if el: return el
            except Exception:
                pass
        for t in textos:
            try:
                el = self.driver.find_element(By.XPATH, f"//a[contains(translate(normalize-space(.),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'), '{t.lower()}')]")
                if el: return el
            except Exception:
                pass
        return None


# -------------------------------
# Execução por CLI (opcional)
# -------------------------------
if __name__ == "__main__":
    import argparse

    ap = argparse.ArgumentParser(description="Consulta CCT/ACT no Sistema Mediador (MTE) e extrai regras de jornada/escala.")
    ap.add_argument("--cnpj", help="CNPJ da empresa (com ou sem pontuação)")
    ap.add_argument("--mes", type=int, default=None, help="Mês de referência (1-12)")
    ap.add_argument("--ano", type=int, default=None, help="Ano de referência (ex.: 2025)")
    ap.add_argument("--sindicato", type=str, default=None, help="Nome (ou parte) do sindicato laboral")
    ap.add_argument("--preferir", type=str, default=None, choices=["CCT","ACT"], help="Preferir CCT ou ACT, se ambos cobrirem a vigência")
    ap.add_argument("--download_dir", type=str, default="./downloads_mediador", help="Pasta para salvar PDFs")
    ap.add_argument("--headless", action="store_true", help="Rodar navegador em modo headless")

    args = ap.parse_args()

    if not args.cnpj:
        ap.error("Informe --cnpj")

    scraper = MediadorScraper(download_dir=args.download_dir, headless=args.headless)
    try:
        resultados = scraper.buscar_por_cnpj(args.cnpj)
        if not resultados:
            print("Nenhum instrumento encontrado para o CNPJ informado.")
            raise SystemExit(0)

        if args.mes and args.ano:
            escolhido = MediadorScraper.selecionar_aplicavel(resultados, mes=args.mes, ano=args.ano, sindicato_nome=args.sindicato, preferir_tipo=args.preferir)
        else:
            # pega o mais recente
            escolhido = sorted(resultados, key=lambda x: (x.vigencia_inicio or datetime.min), reverse=True)[0]

        if not escolhido:
            print("Não foi possível selecionar um instrumento aplicável com os filtros fornecidos.")
            raise SystemExit(0)

        print("Selecionado:\n", json.dumps({k:v for k,v in asdict(escolhido).items() if k != 'link_pdf'}, default=str, ensure_ascii=False, indent=2))

        pdf_path = scraper.baixar_pdf(escolhido)
        print(f"PDF salvo em: {pdf_path}")

        regras = scraper.extrair_regras_pdf(pdf_path)
        print("Regras extraídas:\n", json.dumps(regras, ensure_ascii=False, indent=2))

    finally:
        scraper.fechar()
